<a href="https://colab.research.google.com/github/ThisuriCham/Pytorch/blob/master/training_classifier_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip3 install torchvision

**Augmentations**

In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

**Downloading and loading data sets**

In [12]:
trainset = tv.datasets.CIFAR10(root = './data',train=True,transform=True,download=True)
trainLoader = DataLoader(trainset,batch_size=4,shuffle=True,num_workers=0)

testset = tv.datasets.CIFAR10(root = './data',train=False,transform=True,download=True)
testloader = DataLoader(testset,batch_size=4,shuffle =False,num_workers=0)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


**Display samples**

In [0]:
import matplotlib.pyplot as plt
import numpy as np


In [0]:
def imshow(img):
  img = img / 2 + 0.5
  npimg = img.numpy() 
  plt.figure(figsize=(20,20))
  plt.imshow(np.transpose(npimg, (1,2,0)))
  

dataiter = iter(trainLoader)
images, labels = dataiter.next()

imshow(tv.utils.make_grid(images))
print(''.join('%5s'%classes[labels[j]] for j in range(4)))


In [24]:
trainset.class_to_idx

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

**Define a CNN**

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.conv2 = nn.Conv2d(6,16,5)
    self.pool = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = CNN()
    